# Cleaning and analyzing - contract processes
This notebook will be used for cleaning and analyzing the data scraped from [Sercop's search page](https://modulocomprascorporativas.compraspublicas.gob.ec/ProcesoContratacion/compras/PC/buscarProceso.cpe?sg=1#), scraped with the previuos notebook. 

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/scraped_results_2025-07-09_12-47.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986 entries, 0 to 985
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   codigo             986 non-null    object
 1   detalle_url        986 non-null    object
 2   entidad            986 non-null    object
 3   objeto             986 non-null    object
 4   estado             986 non-null    object
 5   ubicacion          986 non-null    object
 6   presupuesto        986 non-null    object
 7   fecha_publicacion  986 non-null    object
 8   semestre           986 non-null    object
dtypes: object(9)
memory usage: 69.5+ KB


In [4]:
df = df.drop(columns="semestre")

In [5]:
df[df["objeto"].str.contains('"')]

,codigo,detalle_url,entidad,objeto,estado,ubicacion,presupuesto,fecha_publicacion
826,SICM2-534-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,"""DCI: HIERRO, MULTIVITAMINAS Y MINERALES: HIER...",Desierto,PICHINCHA / QUITO,$0.04104,2023-10-06 08:00:00
901,SICM2-345-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,"\""DCI: COMPLEJO B: TIAMINA VITAMINA B1 - PIRID...",Verificación de documentación habilitante otro...,PICHINCHA / QUITO,$0.15161,2023-08-29 08:00:00
911,SICM2-475-2022,https://modulocomprascorporativas.compraspubli...,SERVICIO NACIONAL DE CONTRATACION PUBLICA,"\""DCI: COMPLEJO B: TIAMINA VITAMINA B1 - PIRID...",Verificación de documentación habilitante otro...,PICHINCHA / QUITO,$0.98436,2023-08-29 08:00:00


In [6]:
df["provincia"] = df["ubicacion"].str.split("/").str[0].str.lower().str.strip()
df["canton"] = df["ubicacion"].str.split("/").str[1].str.lower().str.strip()

In [7]:
df = df.drop(columns="ubicacion")

In [8]:
df["objeto"] = (
    df["objeto"].str.replace("\\", "", regex=False)
        .str.replace('"', '')
        .str.replace("TICA : ", "TICA: ")
        .str.replace("TRACIÓN : ", "TRACIÓN: ")
        .str.replace("MERCIAL : ", "MERCIAL: ")
        .str.replace("-CONCEN", "- CONCEN")
        .str.replace("-PRESEN", "- PRESEN")
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
)

In [9]:
df["presupuesto"] = df["presupuesto"].str.replace("$", "").str.replace(",", "").astype(float)

In [10]:
df["principio_activo"] = (df["objeto"]
    .str.split("DCI: ").str[1]
    .str.split("- FOR").str[0]
    .str.lower()
    .str.strip()
                         )

df["forma_farmaceutica"] = (df["objeto"]
    .str.split("TICA:").str[1]
    .str.split("- CON").str[0]
    .str.lower()
    .str.strip()
                           )

df["concentracion"] = (df["objeto"]
    .str.split("TRACIÓN:").str[1]
    .str.split("- PRE").str[0]
    .str.lower()
    .str.strip()
                      )

df["presentacion_comercial"] = (df["objeto"]
    .str.split("CIAL:").str[1]
    .str.lower()
    .str.strip()
                               )

In [11]:
df = df.rename(columns={"objeto": "item"})

In [12]:
df.columns

Index(['codigo', 'detalle_url', 'entidad', 'item', 'estado', 'presupuesto',
       'fecha_publicacion', 'provincia', 'canton', 'principio_activo',
       'forma_farmaceutica', 'concentracion', 'presentacion_comercial'],
      dtype='object')

In [13]:
df["forma_farmaceutica"] = (df["forma_farmaceutica"]
    .str.replace("-", "")
    .str.replace("oralpolvo", "oral polvo")
    .str.replace(r"\s+", " ", regex=True)
                           )

df["principio_activo"] = (df["principio_activo"]
    .str.replace("-", "")
    .str.replace("/", "")
    .str.replace(r"\s+", " ", regex=True)
                         )

In [14]:
df = df[['codigo', 'detalle_url', 'item', 'principio_activo', 'forma_farmaceutica', 'concentracion',
         'presentacion_comercial', 'estado', 'presupuesto', 'provincia', 'canton', 'fecha_publicacion']]

In [15]:
df.head(1)

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion
0,SICM-499-2022,https://modulocomprascorporativas.compraspubli...,DCI: LEVONORGESTREL - FORMA FARMACÉUTICA: SÓLI...,levonorgestrel,sólido parenteral implante subdérmico,150 mg -2 varillas de 75 mg,caja x implante -s- de 75 mg c/u + trocar -es.,Desierto,62.316,pichincha,quito,2022-06-23 08:00:00


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986 entries, 0 to 985
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   codigo                  986 non-null    object 
 1   detalle_url             986 non-null    object 
 2   item                    986 non-null    object 
 3   principio_activo        986 non-null    object 
 4   forma_farmaceutica      986 non-null    object 
 5   concentracion           986 non-null    object 
 6   presentacion_comercial  986 non-null    object 
 7   estado                  986 non-null    object 
 8   presupuesto             986 non-null    float64
 9   provincia               986 non-null    object 
 10  canton                  986 non-null    object 
 11  fecha_publicacion       986 non-null    object 
dtypes: float64(1), object(11)
memory usage: 92.6+ KB


In [115]:
# Source: https://compracorporativasalud.compraspublicas.gob.ec/SICM/?page_id=184
portfolio = pd.read_excel("PORTAFOLIO-DE-PUBLICACIÓN-CONSOLIDADO-15-08-2022.xlsx", skiprows=3)

In [21]:
portfolio["NOMBRE GENERICO"] = (portfolio["NOMBRE GENERICO"]
    .str.replace("(", "")
    .str.replace(")", "")
    .str.lower()
    .str.strip()
                               )

portfolio["DESCRIPCIÓN DEL GRUPO ANATÓMICO"] = (portfolio["DESCRIPCIÓN DEL GRUPO ANATÓMICO"]
    .str.lower()
    .str.strip()
                               )

In [22]:
portfolio_clean = portfolio.drop_duplicates(subset=["NOMBRE GENERICO"])

In [23]:
df = pd.merge(
    df, 
    portfolio_clean[["NOMBRE GENERICO", "DESCRIPCIÓN DEL GRUPO ANATÓMICO"]], 
    left_on="principio_activo", 
    right_on="NOMBRE GENERICO", 
    how="left"
).drop(columns=["NOMBRE GENERICO"])

df = df.rename(columns={"DESCRIPCIÓN DEL GRUPO ANATÓMICO": "descripcion"})

In [25]:
df.head()

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,descripcion
0,SICM-499-2022,https://modulocomprascorporativas.compraspubli...,DCI: LEVONORGESTREL - FORMA FARMACÉUTICA: SÓLI...,levonorgestrel,sólido parenteral implante subdérmico,150 mg -2 varillas de 75 mg,caja x implante -s- de 75 mg c/u + trocar -es.,Desierto,62.316,pichincha,quito,2022-06-23 08:00:00,sistema genitourinario y hormonas sexuales
1,SICM-514-2022,https://modulocomprascorporativas.compraspubli...,DCI: PREDNISOLONA - FORMA FARMACÉUTICA: SÓLIDO...,prednisolona,sólido oral,20 mg,caja x blíster/ristra,Desierto,0.060,pichincha,quito,2022-06-23 08:00:00,preparados hormonales sistemicos
2,SICM-500-2022,https://modulocomprascorporativas.compraspubli...,DCI: LIDOCAÍNA SIN EPINEFRINA - FORMA FARMACÉU...,lidocaína sin epinefrina,sólido cutáneo parche transdérmico,5 %,caja x parche/parches,Adjudicado oferente ganador,5.000,pichincha,quito,2022-06-23 08:00:00,sistema nervioso
3,SICM-515-2022,https://modulocomprascorporativas.compraspubli...,DCI: PROGESTERONA - FORMA FARMACÉUTICA: SÓLIDO...,progesterona,sólido oral,100 mg,caja x blíster/ristra,Desierto,0.230,pichincha,quito,2022-06-23 08:00:00,sistema genitourinario y hormonas sexuales
4,SICM-516-2022,https://modulocomprascorporativas.compraspubli...,DCI: RITUXIMAB - FORMA FARMACÉUTICA: LÍQUIDO P...,rituximab,líquido parenteral,"1400 mg/11,7 ml","caja x vial x 11,7 ml",Desierto,1406.370,pichincha,quito,2022-06-23 08:00:00,agentes antineoplasicos e inmunomodulares


In [26]:
df.to_csv("cleaned_processes.csv", index=False)

## Analysis

For this analysis, it is important to distinguish between the different variables:
- The "item" variable refers to the object of each procurement process. It includes a unique combination of the active ingredient, pharmaceutical form, concentration, and commercial presentation.
- The variables 'principio_activo', 'forma_farmaceutica', 'concentracion','presentacion_comercial' are derived from "item", but will be used separately for the purposes of the analysis below.

In [17]:
#How many purchsings processes have been launched since 2022?
len(df)

986

In [18]:
#How many items have had at least one process launched
df["item"].nunique()

828

In [20]:
# How many different active principles had at least one process launched
df["principio_activo"].nunique()

364

In [24]:
principios = df["principio_activo"].unique().tolist()


AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [30]:
#Which medicines have more than 1 process launched?
df["item"].value_counts().head().to_frame()

,count
item,
"DCI: FENITOÍNA - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 100 MG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA/FRASCO DOSIS PERSONAL, NO ENVASE HOSPITALARIO",3
DCI: MISOPROSTOL - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 200 MCG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA,3
DCI: ALBÚMINA HUMANA - FORMA FARMACÉUTICA: LÍQUIDO PARENTERAL - CONCENTRACIÓN: 20 % - PRESENTACIÓN COMERCIAL: CAJA X FRASCO X 50 ML,2
DCI: RISPERIDONA - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 1 MG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA,2
DCI: GENTAMICINA - FORMA FARMACÉUTICA: LÍQUIDO PARENTERAL - CONCENTRACIÓN: 80 MG/ML - PRESENTACIÓN COMERCIAL: CAJA X AMPOLLA-S- X 2 ML,2


In [31]:
#Check the processes for Misoprostol launched 3 times
df[df["item"] == "DCI: MISOPROSTOL - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 200 MCG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA"]

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,descripcion
147,SICM-391-2022,https://modulocomprascorporativas.compraspubli...,DCI: MISOPROSTOL - FORMA FARMACÉUTICA: SÓLIDO ...,misoprostol,sólido oral,200 mcg,caja x blíster/ristra,Desierto,0.42000,pichincha,quito,2022-05-27 08:00:00,sistema genitourinario y hormonas sexuales
899,SICM2-391-2022-A,https://modulocomprascorporativas.compraspubli...,DCI: MISOPROSTOL - FORMA FARMACÉUTICA: SÓLIDO ...,misoprostol,sólido oral,200 mcg,caja x blíster/ristra,Verificación de documentación habilitante otro...,0.43094,pichincha,quito,2023-08-29 08:00:00,sistema genitourinario y hormonas sexuales
972,SICM2-391-2022,https://modulocomprascorporativas.compraspubli...,DCI: MISOPROSTOL - FORMA FARMACÉUTICA: SÓLIDO ...,misoprostol,sólido oral,200 mcg,caja x blíster/ristra,Cancelado,0.04309,pichincha,quito,2023-07-21 08:00:00,sistema genitourinario y hormonas sexuales


In [32]:
#Check the processes for Fenitoína launched 3 times
df[df["item"] == "DCI: FENITOÍNA - FORMA FARMACÉUTICA: SÓLIDO ORAL - CONCENTRACIÓN: 100 MG - PRESENTACIÓN COMERCIAL: CAJA X BLÍSTER/RISTRA/FRASCO DOSIS PERSONAL, NO ENVASE HOSPITALARIO"]

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,descripcion
651,SICM-569-2022,https://modulocomprascorporativas.compraspubli...,DCI: FENITOÍNA - FORMA FARMACÉUTICA: SÓLIDO OR...,fenitoína,sólido oral,100 mg,"caja x blíster/ristra/frasco dosis personal, n...",Desierto,0.04500,pichincha,quito,2022-08-09 08:00:00,sistema nervioso
728,SICM-569-2022-A,https://modulocomprascorporativas.compraspubli...,DCI: FENITOÍNA - FORMA FARMACÉUTICA: SÓLIDO OR...,fenitoína,sólido oral,100 mg,"caja x blíster/ristra/frasco dosis personal, n...",Desierto,0.04500,pichincha,quito,2023-02-24 08:00:00,sistema nervioso
824,SICM2-569-2022,https://modulocomprascorporativas.compraspubli...,DCI: FENITOÍNA - FORMA FARMACÉUTICA: SÓLIDO OR...,fenitoína,sólido oral,100 mg,"caja x blíster/ristra/frasco dosis personal, n...",Verificación de documentación habilitante otro...,0.04617,pichincha,quito,2023-10-06 08:00:00,sistema nervioso


In [33]:
# Which kind of status of processes
df["estado"].value_counts().to_frame()

,count
estado,
Desierto,335
Adjudicado oferente ganador,274
Adjudicado otros oferentes,140
Verificación de documentación habilitante otros oferentes,137
Verificación de documentación habilitante oferente ganador,52
Por Adjudicar oferente ganador,22
Por Adjudicar otros oferentes,18
Suspendido,7
Cancelado,1


In [34]:
# Awarded contracts
len(df[df["estado"].str.contains("Adjudicado")])

414

In [35]:
# Failed contracts
len(df[~df["estado"].str.contains("Adjudicado")])

572

In [36]:
df["descripcion"].value_counts().to_frame()

,count
descripcion,
antiinfecciosos para uso sistemico,189
sistema nervioso,166
agentes antineoplasicos e inmunomodulares,153
sangre y organos formadores de sangre,86
sistema cardiovascular,69
tracto alimentario y metabolismo,58
preparados hormonales sistemicos,52
dermatologicos,50
sistema genitourinario y hormonas sexuales,36


------

### Items that got a contract

In [37]:
procesos_adjudicados = df[(df["estado"] == "Adjudicado oferente ganador") | (df["estado"] == "Adjudicado otros oferentes")]
len(procesos_adjudicados)

414

In [38]:
procesos_adjudicados.head(2)

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,descripcion
2,SICM-500-2022,https://modulocomprascorporativas.compraspubli...,DCI: LIDOCAÍNA SIN EPINEFRINA - FORMA FARMACÉU...,lidocaína sin epinefrina,sólido cutáneo parche transdérmico,5 %,caja x parche/parches,Adjudicado oferente ganador,5.00,pichincha,quito,2022-06-23 08:00:00,sistema nervioso
9,SICM-519-2022,https://modulocomprascorporativas.compraspubli...,DCI: LOPINAVIR + RITONAVIR - FORMA FARMACÉUTIC...,lopinavir + ritonavir,sólido oral,200 mg + 50 mg,"caja x frasco dosis personal, no envase hospit...",Adjudicado otros oferentes,0.27,pichincha,quito,2022-06-23 08:00:00,antiinfecciosos para uso sistemico


In [39]:
procesos_adjudicados["principio_activo"].nunique()

259

In [40]:
procesos_adjudicados["item"].nunique()

414

In [41]:
procesos_adjudicados["principio_activo"].value_counts().to_frame().head()

,count
principio_activo,
paracetamol,8
cloruro de sodio,5
levotiroxina sódica,5
risperidona,5
cotrimoxazol sulfametoxazol + trimetoprima,4


In [42]:
procesos_adjudicados["descripcion"].value_counts()

descripcion
sistema nervioso                                         69
antiinfecciosos para uso sistemico                       64
agentes antineoplasicos e inmunomodulares                64
sangre y organos formadores de sangre                    44
sistema cardiovascular                                   37
tracto alimentario y metabolismo                         22
preparados hormonales sistemicos                         22
dermatologicos                                           19
sistema respiratorio                                     17
sistema genitourinario y hormonas sexuales               15
organos de los sentidos                                  14
sistema musculo esqueletico                              10
varios                                                    9
productos antiparasitarios, insecticidas y repelentes     5
Name: count, dtype: int64

In [43]:
procesos_adjudicados[procesos_adjudicados["principio_activo"] == "paracetamol"]

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,descripcion
378,SICM-136-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,paracetamol,líquido oral,160 mg/5 ml,caja x frasco x 60 ml - mínimo - con dosificador,Adjudicado oferente ganador,0.61005,pichincha,quito,2022-03-29 08:00:00,sistema nervioso
379,SICM-137-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,paracetamol,líquido parenteral,10 mg/ml,caja x vial-es x 50 ml,Adjudicado oferente ganador,1.00000,pichincha,quito,2022-03-29 08:00:00,sistema nervioso
380,SICM-138-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,paracetamol,líquido parenteral,10 mg/ml,caja x vial-es x 100 ml,Adjudicado otros oferentes,0.69000,pichincha,quito,2022-03-29 08:00:00,sistema nervioso
407,SICM-135-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,paracetamol,líquido oral,150 mg/5 ml,caja x frasco x 60 ml - mínimo con dosificador,Adjudicado oferente ganador,0.70550,pichincha,quito,2022-03-29 08:00:00,sistema nervioso
475,SICM-051-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,paracetamol,líquido oral,100 mg/ml,caja x frasco gotero x 15 ml -mínimo-,Adjudicado oferente ganador,0.24246,pichincha,quito,2022-03-24 08:00:00,sistema nervioso
518,SICM-024-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: SÓLIDO ...,paracetamol,sólido oral,500 mg,caja x blíster/ristra,Adjudicado oferente ganador,0.00683,pichincha,quito,2022-03-23 08:00:00,sistema nervioso
942,SICM2-134-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: LÍQUIDO...,paracetamol,líquido oral,120 mg/5 ml,caja x frasco x 60 ml mínimo- con dosificador,Adjudicado oferente ganador,0.51302,pichincha,quito,2023-07-26 08:00:00,sistema nervioso
956,SICM2-139-2022,https://modulocomprascorporativas.compraspubli...,DCI: PARACETAMOL - FORMA FARMACÉUTICA: SÓLIDO ...,paracetamol,sólido rectal,300 mg,caja x blíster/ristra,Adjudicado oferente ganador,0.17135,pichincha,quito,2023-07-21 08:00:00,sistema nervioso


In [45]:
procesos_adjudicados.sort_values(by="presupuesto", ascending=False).head()

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,descripcion
34,SICM-472-2022,https://modulocomprascorporativas.compraspubli...,DCI: CETUXIMAB - FORMA FARMACÉUTICA: LÍQUIDO P...,cetuximab,líquido parenteral,5 mg/ml,caja x vial-es x 100 ml,Adjudicado oferente ganador,1817.74727,pichincha,quito,2022-06-22 08:00:00,agentes antineoplasicos e inmunomodulares
554,SICM-652-2022,https://modulocomprascorporativas.compraspubli...,DCI: BASILIXIMAB - FORMA FARMACÉUTICA: SÓLIDO ...,basiliximab,sólido parenteral,20 mg,caja x vial-es- con diluyente - caja x vial-es-,Adjudicado oferente ganador,1334.44900,pichincha,quito,2022-08-15 08:00:00,agentes antineoplasicos e inmunomodulares
80,SICM-444-2022,https://modulocomprascorporativas.compraspubli...,DCI: OCTREOTIDA - FORMA FARMACÉUTICA: SÓLIDO P...,octreotida,sólido parenteral,30 mg,caja x vial con diluyente,Adjudicado otros oferentes,1200.00000,pichincha,quito,2022-06-21 08:00:00,preparados hormonales sistemicos
377,SICM-168-2022,https://modulocomprascorporativas.compraspubli...,DCI: TRASTUZUMAB - FORMA FARMACÉUTICA: SÓLIDO ...,trastuzumab,sólido parenteral,440 mg,caja x vial con diluyente,Adjudicado oferente ganador,875.00000,pichincha,quito,2022-03-30 08:00:00,agentes antineoplasicos e inmunomodulares
418,SICM-113-2022,https://modulocomprascorporativas.compraspubli...,DCI: OCTREOTIDA - FORMA FARMACÉUTICA: SÓLIDO P...,octreotida,sólido parenteral,20 mg,caja x vial con diluyente,Adjudicado oferente ganador,550.00000,pichincha,quito,2022-03-28 08:00:00,preparados hormonales sistemicos


In [46]:
#Export so we can scrape the contract information
procesos_adjudicados.to_csv("adjudicados.csv", index=False)

---------

### Medicine that never got a contract
From the previous analysis, we can see there are items that never got a contract, even if they have several purchasing processes launched.

In [51]:
#Items that have an assigned supplier (adjudicado)
items_adjudicados = (df.groupby("item")["estado"]
    .apply(lambda x: x.isin(["Adjudicado oferente ganador", "Adjudicado otros oferentes"])
    .any())
              )

items_adjudicados.value_counts()

estado
True     414
False    414
Name: count, dtype: int64

In [54]:
# Items that never got a contract
items_no_adjudicados = items_adjudicados[~items_adjudicados].index.to_list()
items_no_adjudicados = pd.DataFrame(items_no_adjudicados, columns=["item"])

In [55]:
len(items_no_adjudicados)

414

In [56]:
items_no_adjudicados["principio_activo"] = (items_no_adjudicados["item"]
    .str.split("DCI: ").str[1]
    .str.split("- FOR").str[0]
    .str.lower()
    .str.strip()
                                    )

items_no_adjudicados["forma_farmaceutica"] = (items_no_adjudicados["item"]
    .str.split("TICA:").str[1]
    .str.split("- CON").str[0]
    .str.lower()
    .str.strip()
                                      )

items_no_adjudicados["concentracion"] = (items_no_adjudicados["item"]
    .str.split("TRACIÓN:").str[1]
    .str.split("- PRE").str[0]
    .str.lower()
    .str.strip()
                                 )

items_no_adjudicados["presentacion_comercial"] = (items_no_adjudicados["item"]
    .str.split("CIAL:").str[1]
    .str.lower()
    .str.strip()
                                          )

,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial
0,DCI: ACICLOVIR - FORMA FARMACÉUTICA: LÍQUIDO O...,aciclovir,líquido oral,200 mg/5 ml,caja x frasco x 100 ml -mínimo- con dosificador
1,DCI: ACICLOVIR - FORMA FARMACÉUTICA: SEMISÓLID...,aciclovir,semisólido oftálmico,3 %,caja x tubo x 5 g


In [57]:
items_no_adjudicados = pd.merge(
    items_no_adjudicados, 
    portfolio_clean[["NOMBRE GENERICO", "DESCRIPCIÓN DEL GRUPO ANATÓMICO"]], 
    left_on="principio_activo", 
    right_on="NOMBRE GENERICO", 
    how="left"
).drop(columns=["NOMBRE GENERICO"])

items_no_adjudicados = items_no_adjudicados.rename(columns={"DESCRIPCIÓN DEL GRUPO ANATÓMICO": "descripcion"})

items_no_adjudicados.head(2)

,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,descripcion
0,DCI: ACICLOVIR - FORMA FARMACÉUTICA: LÍQUIDO O...,aciclovir,líquido oral,200 mg/5 ml,caja x frasco x 100 ml -mínimo- con dosificador,antiinfecciosos para uso sistemico
1,DCI: ACICLOVIR - FORMA FARMACÉUTICA: SEMISÓLID...,aciclovir,semisólido oftálmico,3 %,caja x tubo x 5 g,antiinfecciosos para uso sistemico


In [61]:
# Active principles that have failed processes the most
items_no_adjudicados["principio_activo"].value_counts().head().to_frame()

,count
principio_activo,
morfina,8
oxicodona,7
levotiroxina sódica,7
metronidazol,7
leuprolide,6


In [67]:
items_no_adjudicados["descripcion"].value_counts().to_frame()

,count
descripcion,
antiinfecciosos para uso sistemico,88
sistema nervioso,66
agentes antineoplasicos e inmunomodulares,58
tracto alimentario y metabolismo,27
sangre y organos formadores de sangre,26
sistema cardiovascular,22
dermatologicos,20
preparados hormonales sistemicos,19
varios,12


In [62]:
#Example - what kinds of morfine didn't get a contract
items_no_adjudicados[items_no_adjudicados["item"].str.contains("MORFINA")]

,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,descripcion
291,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO ORA...,morfina,líquido oral,2 mg/ml,caja x frasco x 60 ml mínimo - con dosificador,sistema nervioso
292,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO ORA...,morfina,líquido oral,2 mg/ml,caja x frasco x 60 ml mínimo- con dosificador,sistema nervioso
293,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO ORA...,morfina,líquido oral,20 mg/ml,caja x frasco x 60 ml mínimo - con dosificador,sistema nervioso
294,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO ORA...,morfina,líquido oral,20 mg/ml,caja x frasco x 60 ml mínimo con dosificador,sistema nervioso
295,DCI: MORFINA - FORMA FARMACÉUTICA: SÓLIDO ORAL...,morfina,sólido oral -liberación prolongada-,30 mg,caja x blíster/ristra,sistema nervioso
296,DCI: MORFINA - FORMA FARMACÉUTICA: SÓLIDO ORAL...,morfina,sólido oral liberación prolongada,10 mg,caja x blíster/ristra,sistema nervioso
297,DCI: MORFINA - FORMA FARMACÉUTICA: SÓLIDO ORAL...,morfina,sólido oral liberación prolongada,30 mg,caja x blíster/ristra,sistema nervioso
298,DCI: MORFINA - FORMA FARMACÉUTICA: SÓLIDO ORAL...,morfina,sólido oral liberación prolongada,60 mg,caja x blíster/ristra,sistema nervioso


In [63]:
# Is there morfine that DID get a contract?
df[(df["principio_activo"] == "morfina") & (df["estado"].str.contains("Adjudicado"))]

,codigo,detalle_url,item,principio_activo,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,descripcion
401,SICM-129-2022,https://modulocomprascorporativas.compraspubli...,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO PAR...,morfina,líquido parenteral,10 mg/ml,caja x ampolla - s x 1 ml,Adjudicado oferente ganador,0.73013,pichincha,quito,2022-03-29 08:00:00,sistema nervioso
560,SICM-658-2022,https://modulocomprascorporativas.compraspubli...,DCI: MORFINA - FORMA FARMACÉUTICA: LÍQUIDO PAR...,morfina,líquido parenteral,20 mg/ml,caja x ampolla-s- x 1 ml,Adjudicado oferente ganador,0.85964,pichincha,quito,2022-08-15 08:00:00,sistema nervioso
720,SICM-441-2022-A,https://modulocomprascorporativas.compraspubli...,DCI: MORFINA - FORMA FARMACÉUTICA: SÓLIDO ORAL...,morfina,sólido oral,10 mg,caja x blíster/ristra,Adjudicado oferente ganador,0.44587,pichincha,quito,2023-02-24 08:00:00,sistema nervioso


In [64]:
items_no_adjudicados.columns

Index(['item', 'principio_activo', 'forma_farmaceutica', 'concentracion',
       'presentacion_comercial', 'descripcion'],
      dtype='object')

In [65]:
# Is there active principles that, in any form, didn't get any contract?
adjudicados_principles = (df.groupby("principio_activo")["estado"]
    .apply(lambda x: x.isin(["Adjudicado oferente ganador", "Adjudicado otros oferentes"])
    .any())
                         )

adjudicados_principles.value_counts()

estado
True     259
False    105
Name: count, dtype: int64

In [66]:
never_adjudicados_principles = adjudicados_principles[~adjudicados_principles].index.to_list()
len(never_adjudicados_principles)

105

In [68]:
items_no_adjudicados["tiene_otro_contrato"] = items_no_adjudicados["principio_activo"].apply(lambda x: "No" if x in never_adjudicados_principles else "Si")

In [69]:
items_no_adjudicados.to_csv("items_sin_contrato.csv", index=False)

-----

### Details for medicine that never got a contract

In [103]:
no_contract_principles = pd.DataFrame(never_adjudicados_principles, columns=["principio_activo"])
len(no_contract_principles)

105

In [104]:
no_contract_principles = pd.merge(
    no_contract_principles,
    df,
    on="principio_activo",
    how="left"
)

In [106]:
no_contract_principles["estado"].value_counts().to_frame()

,count
estado,
Desierto,143
Verificación de documentación habilitante otros oferentes,74
Verificación de documentación habilitante oferente ganador,32
Por Adjudicar oferente ganador,11
Por Adjudicar otros oferentes,7
Suspendido,4
Cancelado,1


In [107]:
no_contract_principles["descripcion"].value_counts().to_frame()

,count
descripcion,
agentes antineoplasicos e inmunomodulares,58
antiinfecciosos para uso sistemico,38
sistema nervioso,37
tracto alimentario y metabolismo,28
sangre y organos formadores de sangre,20
sistema cardiovascular,17
preparados hormonales sistemicos,17
dermatologicos,13
sistema genitourinario y hormonas sexuales,11


In [108]:
no_contract_principles["descripcion"].isna().value_counts()

descripcion
False    269
True       3
Name: count, dtype: int64

In [109]:
no_contract_principles[no_contract_principles["descripcion"].isna()]

,principio_activo,codigo,detalle_url,item,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,descripcion
95,"hierro, multivitaminas y minerales: hierro zin...",SICM-534-2022,https://modulocomprascorporativas.compraspubli...,"DCI: HIERRO, MULTIVITAMINAS Y MINERALES: HIERR...",sólido oral polvo,"* 12,5 mg* 5 mg* 300 mcg* 160 mcg* 30 mg",caja x sobres/sachets,Desierto,0.04000,pichincha,quito,2022-08-08 08:00:00,NaN
96,"hierro, multivitaminas y minerales: hierro zin...",SICM2-534-2022,https://modulocomprascorporativas.compraspubli...,"DCI: HIERRO, MULTIVITAMINAS Y MINERALES: HIERR...",sólido oral polvo,"* 12,5 mg* 5 mg* 300 mcg* 160 mcg* 30 mg",caja x sobres/sachets,Desierto,0.04104,pichincha,quito,2023-10-06 08:00:00,NaN
271,ácido trans retinoico tretinoína atra,SICM-679-2022,https://modulocomprascorporativas.compraspubli...,DCI: ÁCIDO TRANS RETINOICO -TRETINOÍNA- - ATRA...,sólido oral,10 mg,"caja x blíster/ristra/frasco dosis personal, n...",Verificación de documentación habilitante otro...,2.78000,pichincha,quito,2023-02-22 08:00:00,NaN


In [110]:
len(no_contract_principles)

272

In [111]:
no_contract_principles["principio_activo"].nunique()

105

In [112]:
no_contract_principles["item"].nunique()

199

In [113]:
no_contract_principles.head()

,principio_activo,codigo,detalle_url,item,forma_farmaceutica,concentracion,presentacion_comercial,estado,presupuesto,provincia,canton,fecha_publicacion,descripcion
0,agua para irrigación,SICM-464-2022,https://modulocomprascorporativas.compraspubli...,DCI: AGUA PARA IRRIGACIÓN - FORMA FARMACÉUTICA...,líquido parenteral,no definido,bolsa x 3000 ml,Desierto,2.63500,pichincha,quito,2022-06-21 08:00:00,varios
1,agua para irrigación,SICM2-464-2022,https://modulocomprascorporativas.compraspubli...,DCI: AGUA PARA IRRIGACIÓN - FORMA FARMACÉUTICA...,líquido parenteral,no definido,bolsa x 3000 ml,Por Adjudicar oferente ganador,3.09362,pichincha,quito,2023-07-27 08:00:00,varios
2,alopurinol,SICM-301-2022,https://modulocomprascorporativas.compraspubli...,DCI: ALOPURINOL - FORMA FARMACÉUTICA: SÓLIDO O...,sólido oral,100 mg,caja x blíster/ristra,Desierto,0.04544,pichincha,quito,2022-05-24 08:00:00,sistema musculo esqueletico
3,alopurinol,SICM-229-2022,https://modulocomprascorporativas.compraspubli...,DCI: ALOPURINOL - FORMA FARMACÉUTICA:SÓLIDO OR...,sólido oral,300 mg,caja x blíster/ristra,Desierto,0.02525,pichincha,quito,2022-05-03 08:00:00,sistema musculo esqueletico
4,alopurinol,SICM2-301-2022,https://modulocomprascorporativas.compraspubli...,DCI: ALOPURINOL - FORMA FARMACÉUTICA: SÓLIDO O...,sólido oral,100 mg,caja x blíster/ristra,Verificación de documentación habilitante otro...,0.05181,pichincha,quito,2023-07-27 08:00:00,sistema musculo esqueletico


In [114]:
no_contract_principles.to_csv("principios_sin_contrato.csv", index=False)